In [7]:
using CSV, DataFrames
using LinearAlgebra, Random, Statistics
using Convex, SCS

In [8]:
fname = "datasets/tract_dataset.csv"
df = DataFrame(CSV.File(fname));

In [9]:
cols = names(df)
for col in cols
    m = .!isnan.(df[:,col])
    println("$(col) $(sum(.!m))")
end

TRACT 0
single_parent_household 1535
youth_not_in_school 2132
sex_pay_inequity 877
race_pay_inequity 47348
snap_household 1097
snap_vulnerabe_household 1097
no_internet_household 1097
no_smartphone_household 1097
no_car_household 1097
no_plumbing_household 1097
over_60min_commute 1009
wfh_inv 998
biking_commute_inv 998
working_class_workers 1006
unemployment_rate 0
renter_occupied 0
median_income_inv 0
Crimes Against Persons Rate 19433
Crimes Against Property Rate 19433
Crimes Against Society Rate 19433
ALR_VALB 1243
ALR_VALP 1243
lahunv1share 42885
lasnap1share 42885


In [10]:
(n, m) = size(df[!, cols[2:end]])

(85381, 24)

In [11]:
A = Matrix(df[:, cols[2:end]])
M = isnan.(A)

corr = zeros(m,m)
for i in 1:m
    for j in i:m
        mask = (M[:,i] .+ M[:,j]) .== 0
        x = cor(Vector(df[mask ,cols[1+i]]), Vector(df[mask ,cols[1+j]]))
        corr[i, j] = x
        corr[j, i] = x
    end
end

eigs = eigvals(corr)
nfa = sum(eigs .> 1)
k = nfa

7

In [12]:
df_Y = DataFrame(CSV.File("weights/zcta_Y.csv"))
k_init = minimum([size(df_Y)[1], k])
Y_prev = Matrix(df_Y)[1:k_init, :];

In [13]:
MAX_ITERS = 20
residual = zeros(MAX_ITERS);
rng = Xoshiro(42)

X_0 = rand(rng, Float64,(n, k));
X_0 = X_0 ./ (X_0 * ones(k, 1))

Y_0 = rand(rng, Float64,(k, m));
Y_0[1:k_init, :] = Y_prev
H = ones((n, m))
p = 0
constraint = []

A[M] .= 0
H[M] .= 0

step = 2500

for i = 1:MAX_ITERS
    
    if (i % 2) == 0
        Y = Variable((k,m))
        set_value!(Y, Y_0)
        X = X_0
        #constraint = [Y >= 0]
        constraint = [Y[1,:] >= 0]
        #constraint = []
        obj = sum(huber(A - X*Y.*H, .4)) + 12 * norm(Y[2:k,:], 1) + 6 * norm(Y[2:k,:], 2)

        p = minimize(obj, constraint)
        solve!(p, SCS.Optimizer; silent = false)

        if p.status != Convex.MOI.OPTIMAL
            println("Failed to Converge")
        end

        Y_0 = Y.value

        residual[i] = p.optval
        println("$i, $(residual[i])")

    else
        for j in 1:step:n
            if j + step > n
                dim = n - j + 1
            else
                dim = step
            end
                
            X = Variable((dim, k))
            set_value!(X, X_0[j:j+dim-1, :])
            Y = Y_0
            #constraint = [X >= 0, X * ones(k, 1) <= 1]
            constraint = [X[:,1] >= 0, opnorm(X, Inf) <= 1]
            #constraint = []
            obj = sum(huber(A[j:j+dim-1, :] - X*Y.*H[j:j+dim-1, :], .4))

            p = minimize(obj, constraint)
            solve!(p, SCS.Optimizer; silent = false)

            if p.status != Convex.MOI.OPTIMAL
                println("Failed to Converge")
            end

            residual[i] += p.optval
            
            X_0[j:j+dim-1, :] = X.value
        end
         println("$i, $(residual[i])")
    end
end

println()

w = mean(abs.(X_0), dims=1) .* mean(abs.(Y_0), dims=2)'
w = w ./ sum(w)
println(w)

[ Info: [Convex.jl] Compilation finished: 7.58 seconds, 910.523 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1017592, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.09e

[ Info: [Convex.jl] Compilation finished: 0.74 seconds, 348.102 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1019062, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.03e

[ Info: [Convex.jl] Compilation finished: 0.71 seconds, 343.406 MiB of memory allocated
[ Info: [Convex.jl] Compilation finished: 0.47 seconds, 346.231 MiB of memory allocated


...

[ Info: [Convex.jl] Compilation finished: 0.66 seconds, 348.090 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1018950, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.10e

[ Info: [Convex.jl] Compilation finished: 0.46 seconds, 348.965 MiB of memory allocated
[ Info: [Convex.jl] Compilation finished: 0.69 seconds, 349.245 MiB of memory allocated


------------------------------------------------------------------
	       SCS v3.2.7 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 275001, constraints m: 400001
cones: 	  z: primal zero / dual free vars: 60000
	  l: linear vars: 160001
	  q: soc vars: 180000, qsize: 60000
settings: eps_abs: 1.0e-004, eps_rel: 1.0e-004, eps_infeas: 1.0e-007
	  alpha: 1.50, scale: 1.00e-001, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-006
	  acceleration_lookback: 10, acceleration_interval: 10
	  compiled with openmp parallelization enabled
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 1028225, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0|1.45e+001 1.00e+000 1.74e+006 -8.71e+005 1.00e-001 6.05e

[ Info: [Convex.jl] Compilation finished: 0.62 seconds, 345.140 MiB of memory allocated
Excessive output truncated after 527513 bytes.

In [14]:
for i = 1:20
    println("$i, $(residual[i])")
end

1, 56376.56147428147
2, 56318.598681318224
3, 54317.706704358345
4, 55380.83877627776
5, 53786.16494494109
6, 55050.49616075085
7, 53561.506347037
8, 54892.60560492155
9, 53442.39784958992
10, 54802.80188403126
11, 53366.9527156559
12, 54744.29700822051
13, 53314.50336051539
14, 54703.02726587302
15, 53274.783500050566
16, 54671.632060203025
17, 53243.077539865284
18, 54646.72849911044
19, 53217.32287572633
20, 54626.53548382251


In [15]:
w = mean(abs.(X_0), dims=1) .* mean(abs.(Y_0), dims=2)'
w = w ./ sum(w)
println(w)

[0.49906300679422255 0.07654148708327775 0.07901071563609516 0.10240982978967438 0.0955445236489135 0.09379561482739829 0.05363482222041841]


In [16]:
df_Y = DataFrame(Y_0, cols[2:end])
fname = "weights/tract_Y.csv"
CSV.write(fname, df_Y)

"weights/tract_Y.csv"

In [18]:
df_X = df[:, ["TRACT"]]
for i in 1:k
    df_X[:, "$i"] = X_0[:, i]
end
fname = "weights/tract_X.csv"
CSV.write(fname, df_X)

"weights/tract_X.csv"